In [ ]:
%load_ext autoreload
%autoreload 2
import sys
from ridge_utils.DataSequence import DataSequence
import pandas as pd
import os
import matplotlib.pyplot as plt
import cortex
import seaborn as sns
from os.path import join
from collections import defaultdict
import numpy as np
import sasc.viz
import joblib
import dvu
import sys
sys.path.append('../notebooks')
from tqdm import tqdm
from sasc.config import FMRI_DIR, STORIES_DIR, RESULTS_DIR
from neuro.config import repo_dir
from neuro import analyze_helper, viz
from neuro.features.qa_questions import get_questions, get_merged_questions_v3_boostexamples
flatmaps_per_question = __import__('06_flatmaps_per_question')
from neurosynth import term_dict, term_dict_rev
import viz

In [ ]:
from load_coef_flatmaps import _load_coefs_35questions, _load_coefs_individual, _load_coefs_individual_wordrate, _load_coefs_wordrate

In [ ]:
results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/aug12_neurosynth_gemv'
rr_shapley, cols_varied, mets = analyze_helper.load_clean_results(results_dir)

In [ ]:
def _load_coefs_shapley(subject='S02', qa_questions_version='v3_boostexamples_merged'):
    r = rr_shapley
    r = r[r['qa_questions_version'] == qa_questions_version]
    r = r[r['subject'] == subject]
    print(r.shape, rr_shapley.shape)
    row = r.iloc[0]

    if qa_questions_version == 'v3_boostexamples_merged':
        questions = get_merged_questions_v3_boostexamples()
        questions = np.array(questions)[row.weight_enet_mask]
    else:
        questions = get_questions(qa_questions_version)

    flatmaps_shapley = defaultdict(list)
    for i in range(len(r)):
        row = r.iloc[i]
        weights, weights_pc = flatmaps_per_question.get_weights_top(row)
        question_indexes = np.arange(len(questions))[
            row['weight_random_mask'][:len(questions)]]

        for w_idx, q_idx in enumerate(question_indexes):
            flatmaps_shapley[questions[q_idx]].append(weights[w_idx])

    print('lens', [len(w) for w in flatmaps_shapley.values()])

    for q, w in flatmaps_shapley.items():
        flatmaps_shapley[q] = np.vstack(w).mean(axis=0)
    return flatmaps_shapley


subject = 'S03'
df_w_shapley35 = _load_coefs_shapley(
    subject=subject, qa_questions_version='v3_boostexamples_merged')

df_w_shapley_neurosynth = _load_coefs_shapley(
    subject, qa_questions_version='v1neurosynth')

In [ ]:
# subject = 'S02'
for subject in ['S01', 'S02', 'S03']:
    # jointly fitted 35-question model
    df_w_selected35 = _load_coefs_35questions(subject=subject)

    # individually fitted question models
    df_w_individual = _load_coefs_individual(subject=subject)

    # individually fitted question models *with wordrate
    df_w_individual_wordrate = _load_coefs_individual_wordrate(
        subject=subject)

    # wordrate
    df_w_wordrate_alone = _load_coefs_wordrate(subject=subject)

    df_w_shapley35 = _load_coefs_shapley(
        subject, qa_questions_version='v3_boostexamples_merged')

    df_w_shapley_neurosynth = _load_coefs_shapley(
        subject, qa_questions_version='v1neurosynth')

    # collate individual dfs #########################
    # average weights for df_w_selected35 and df_w_individual
    if subject == 'S02':
        df_avg = df_w_selected35.merge(df_w_individual, on='question')
        df_avg['weights'] = df_avg.apply(
            lambda x: np.mean([x['weights_x'], x['weights_y']], axis=0), axis=1)

    df_avg_individual = df_w_individual.merge(
        df_w_individual_wordrate, on='question')
    df_avg_individual['weights'] = df_avg_individual.apply(
        lambda x: np.mean([x['weights_x'], x['weights_y']], axis=0), axis=1)

    df_qa_dict = {
        'selected35': df_w_selected35,
        'individual': df_w_individual,
        'individual_wordrate': df_w_individual_wordrate,
        'wordrate_alone': df_w_wordrate_alone,
        # 'avg': df_avg,
        'shapley_neurosynth': df_w_shapley_neurosynth,
        'shapley35': df_w_shapley35,
        'avg_individual': df_avg_individual
    }
    joblib.dump(df_qa_dict, f'df_qa_dict_{subject}.pkl')